In [ ]:
!pip install duckdb --no-index --find-links=/kaggle/input/polars-and-duckdb/kaggle/working/mysitepackages/duck_pkg
!pip install polars --no-index --find-links=/kaggle/input/polars-and-duckdb/kaggle/working/mysitepackages/polars_pkg
#!pip install fair-esm --no-index --find-links=/kaggle/input/suman-fair-esm/kaggle/working/fair_esm-2.0.0-py3-none-any.whl

In [ ]:
import torch

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import random
import pprint
import os
import duckdb as dd
import polars as pl

In [ ]:
!pip install -U transformers datasets evaluate accelerate timm

In [1]:
#from transformers import AutoTokenizer, EsmForProteinFolding, EsmModel
import transformers

In [3]:
## Run this twice, first time, it will show error
from transformers import AutoTokenizer, EsmForProteinFolding, EsmModel

In [4]:
tokenizer = AutoTokenizer.from_pretrained("facebook/esmfold_v1")
model = EsmModel.from_pretrained("facebook/esmfold_v1")

tokenizer_config.json:   0%|          | 0.00/40.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/121 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/8.44G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/8.44G [00:00<?, ?B/s]

Some weights of EsmModel were not initialized from the model checkpoint at facebook/esmfold_v1 and are newly initialized: ['contact_head.regression.bias', 'contact_head.regression.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [7]:
model.to(device)
model.eval()

EsmModel(
  (embeddings): EsmEmbeddings(
    (word_embeddings): Embedding(33, 2560, padding_idx=1)
    (dropout): Dropout(p=0.0, inplace=False)
  )
  (encoder): EsmEncoder(
    (layer): ModuleList(
      (0-35): 36 x EsmLayer(
        (attention): EsmAttention(
          (self): EsmSelfAttention(
            (query): Linear(in_features=2560, out_features=2560, bias=True)
            (key): Linear(in_features=2560, out_features=2560, bias=True)
            (value): Linear(in_features=2560, out_features=2560, bias=True)
            (dropout): Dropout(p=0.0, inplace=False)
            (rotary_embeddings): RotaryEmbedding()
          )
          (output): EsmSelfOutput(
            (dense): Linear(in_features=2560, out_features=2560, bias=True)
            (dropout): Dropout(p=0.0, inplace=False)
          )
          (LayerNorm): LayerNorm((2560,), eps=1e-05, elementwise_affine=True)
        )
        (intermediate): EsmIntermediate(
          (dense): Linear(in_features=2560, out_feature

In [8]:
import os

output_dir = "/kaggle/working/esm_embeddings"
os.makedirs(output_dir, exist_ok=True)

In [10]:
!pip install biopython

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 31.5 MB/s eta 0:00:00a 0:00:01


In [11]:
from Bio import SeqIO

In [18]:
sequences = SeqIO.parse("/kaggle/input/cafa-6-protein-function-prediction/Train/train_sequences.fasta", "fasta")

In [19]:
train_sequences_list = list(sequences)
len(train_sequences_list)

82404

In [20]:
last_record = train_sequences_list[-1]  # using Python's list tricks
print(last_record.id)
print(repr(last_record.seq))
print(len(last_record))

sp|Q9Y816|YOND_SCHPO
Seq('MITEFIKSFLLFFFLPFFLSMPMIFATLGEFTDDQTHHYSTLPSCDLLLIRGEV...KLN')
142


In [13]:
type(sequences)

Bio.SeqIO.FastaIO.FastaIterator

In [14]:
for record in sequences:
    seq_id = record.id
    sequence = str(record.seq)

    # Tokenize the sequence
    # The tokenizer adds special tokens (CLS and SEP) automatically
    inputs = tokenizer(sequence, return_tensors='pt', padding=True, truncation=True)
    inputs = {k: v.to(device) for k, v in inputs.items()}

    # Generate embeddings
    with torch.no_grad():
        output = model(**inputs)

    # Extract embeddings
    # We usually take the last hidden states
    embeddings = output.last_hidden_state # Shape: (batch_size, sequence_length, hidden_size)

    # To get a single fixed-size embedding for the whole protein (mean pooling), 
    # you can average over the sequence length dimension (excluding CLS/SEP if desired, though often included for simplicity)
    # Here we average over all tokens in the sequence.
    # Alternatively, you can use the representation of the CLS token
    # sequence_level_embedding = output.pooler_output # This only works for certain model configurations
    
    # Mean pooling as an alternative to pooler_output
    # Start from index 1 and end at index -1 to ignore CLS and SEP tokens if they exist, 
    # but the standard practice with ESM is often to include all or use the representation from a specific layer
    # For simplicity, we can do mean pooling across the sequence dimension for now:
    sequence_embeddings = embeddings.mean(dim=1).squeeze().cpu().numpy() # Shape: (hidden_size,)

    # Save the embeddings
    output_path = os.path.join(output_dir, f"{seq_id}.pt")
    torch.save({'embeddings': sequence_embeddings}, output_path) # Save as a dictionary
    print(f"Saved embeddings for {seq_id} to {output_path}")

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Saved embeddings for sp|A0A0C5B5G6|MOTSC_HUMAN to /kaggle/working/esm_embeddings/sp|A0A0C5B5G6|MOTSC_HUMAN.pt
Saved embeddings for sp|A0JNW5|BLT3B_HUMAN to /kaggle/working/esm_embeddings/sp|A0JNW5|BLT3B_HUMAN.pt
Saved embeddings for sp|A0JP26|POTB3_HUMAN to /kaggle/working/esm_embeddings/sp|A0JP26|POTB3_HUMAN.pt
Saved embeddings for sp|A0PK11|CLRN2_HUMAN to /kaggle/working/esm_embeddings/sp|A0PK11|CLRN2_HUMAN.pt
Saved embeddings for sp|A1A4S6|RHG10_HUMAN to /kaggle/working/esm_embeddings/sp|A1A4S6|RHG10_HUMAN.pt
Saved embeddings for sp|A1A519|F170A_HUMAN to /kaggle/working/esm_embeddings/sp|A1A519|F170A_HUMAN.pt
Saved embeddings for sp|A1L190|SYCE3_HUMAN to /kaggle/working/esm_embeddings/sp|A1L190|SYCE3_HUMAN.pt
Saved embeddings for sp|A1L3X0|ELOV7_HUMAN to /kaggle/working/esm_embeddings/sp|A1L3X0|ELOV7_HUMAN.pt
Saved embeddings for sp|A1X283|SPD2B_HUMAN to /kaggle/working/esm_embeddings/sp|A1X283|SPD2B_HUMAN.pt
Saved embeddings for sp|A2A2Y4|FRMD3_HUMAN to /kaggle/working/esm_embeddin

OutOfMemoryError: CUDA out of memory. Tried to allocate 3.49 GiB. GPU 0 has a total capacity of 15.89 GiB of which 743.12 MiB is free. Process 4822 has 15.16 GiB memory in use. Of the allocated memory 14.50 GiB is allocated by PyTorch, and 379.70 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
from zipfile import ZipFile

dirName = '/kaggle/working'
zipName = 'fair-esm-suman.zip'

# Create a ZipFile Object
with ZipFile(zipName, 'w') as zipObj:
    # Iterate over all the files in directory
    for folderName, subfolders, filenames in os.walk(dirName):
        for filename in filenames:
            if (filename != zipName):
                # create complete filepath of file in directory
                filePath = os.path.join(folderName, filename)
                # Add file to zip
                zipObj.write(filePath)

In [ ]:
!pip install fair-esm --no-index --find-links=/kaggle/input/suman-fair-esm/kaggle/working/fair_esm-2.0.0-py3-none-any.whl

In [ ]:
!python --version

In [ ]:
import esm

In [ ]:
def generate_embeddings_from_fasta_v0_1_0_manual(fasta_file, output_dir):
    """
    Generates and saves protein embeddings from a FASTA file for fair-esm==0.1.0.
    """
    # Load the model and alphabet (adjust model name if needed for v0.1.0)
    model, alphabet = esm.pretrained.esm2_t33_650M_UR50D()
    model = model.eval()
    if torch.cuda.is_available():
        model = model.cuda()
    
    # Use the batch converter
    batch_converter = alphabet.get_batch_converter()
    
    # Manually read sequences from the FASTA file
    data = []
    with open(fasta_file, "r") as f:
        current_label = None
        current_seq = []
        for line in f:
            line = line.strip()
            if line.startswith(">"):
                if current_label and current_seq:
                    data.append((current_label, "".join(current_seq)))
                current_label = line[1:]
                current_seq = []
            else:
                current_seq.append(line)
        if current_label and current_seq:
            data.append((current_label, "".join(current_seq)))

    os.makedirs(output_dir, exist_ok=True)
    
    # Convert data into a batch of tokens
    batch_labels, batch_strs, batch_tokens = batch_converter(data)
    
    # Move tokens to device
    batch_tokens = batch_tokens.to(next(model.parameters()).device)

    with torch.no_grad():
        # Get model embeddings (using 'representations' key for v0.1.0)
        results = model(batch_tokens, repr_layers=[model.num_layers])
        sequence_representations = results["representations"][model.num_layers]

        # Iterate and save embeddings
        for i, label in enumerate(batch_labels):
            filename = os.path.join(output_dir, f"{label.replace('/', '_').replace('|', '_')}.pt")
            
            # Average over sequence length, excluding BOS token
            seq_len = len(batch_strs[i])
            mean_embedding = sequence_representations[i, 1:seq_len + 1].mean(0).cpu().numpy()
            
            # Save embedding
            torch.save({"mean_representations": mean_embedding}, filename)
            print(f"Saved embedding for {label} to {filename}")

In [ ]:
generate_embeddings_from_fasta_v0_1_0_manual("/kaggle/input/cafa-6-protein-function-prediction/Train/train_sequences.fasta" \
                               , "output_embeddings_dir/")